A script to calculate the intersections of multiple multi-polygons

In [1]:
import json
import os
import geopandas as gpd
from collections import defaultdict
from pathlib import Path
from shapely.geometry import mapping, shape
ROOT = Path('../')
ROOT.resolve()

PosixPath('/Users/lukestrange/Code/bus-tracking')

In [ ]:
region = 'north_west'
date = '20240916'
time = '1615'

In [3]:
def round_coords(geom, precision=5):
    '''Function to round coordinates to 5 decimal places'''
    if geom.is_empty:
        return geom
    # Map the geometry to a dict, round coordinates, and return as a new shape
    return shape({
        'type': geom.geom_type,
        'coordinates': _round_recursive(mapping(geom)['coordinates'], precision)
    })

def _round_recursive(coords, precision):
    '''Helper function to round coordinates recursively'''
    if isinstance(coords, (tuple, list)):
        return [
            _round_recursive(c, precision) if isinstance(c, (list, tuple)) else round(c, precision)
            for c in coords
        ]
    return round(coords, precision)

In [4]:
# Define the path to the directory containing the GeoJSON files
directory_path = ROOT / f'data/geojson/{region}/real'
output_file_path = directory_path / f'intersections/wc_{date}_{time}.geojson'

# Initialize a dictionary to hold features grouped by the "time" property
grouped_features = defaultdict(list)

# Loop through each file in the specified directory
for filename in os.listdir(directory_path):
    if filename.endswith('.geojson'):
        file_path = os.path.join(directory_path, filename)
        
        # Read the GeoJSON file into a GeoDataFrame
        gdf = gpd.read_file(file_path)
        
        # Iterate through the features (rows) in the GeoDataFrame
        for index, row in gdf.iterrows():
            time_value = row['time']
            grouped_features[time_value].append(row)

# Create a list to hold combined features for the new GeoDataFrame
combined_features = []

# Construct a new GeoDataFrame with combined features
for time_value, features in grouped_features.items():
    # Start with the geometry of the first feature
    combined_geometry = features[0].geometry
    
    # Validate and clean the initial geometry
    if not combined_geometry.is_valid:
        combined_geometry = combined_geometry.buffer(0)  # Fix invalid geometry
    
    # Iterate through the remaining features and calculate intersections
    for feature in features[1:]:
        current_geometry = feature.geometry
        
        # Validate and clean the current geometry
        if not current_geometry.is_valid:
            current_geometry = current_geometry.buffer(0)  # Fix invalid geometry
            
        # Attempt to find the intersection
        try:
            combined_geometry = combined_geometry.intersection(current_geometry)
        except Exception as e:
            print(f"Error calculating intersection with geometry: {e}")
            continue  # Skip to the next feature if an error occurs
    
    # Only add to combined_features if the intersection is valid
    if not combined_geometry.is_empty and combined_geometry.is_valid:
        # Round the geometry coordinates to 5 decimal places
        rounded_geometry = round_coords(combined_geometry, precision=5)
        # Create a new feature with combined geometry and the corresponding time
        combined_feature = {
            'geometry': rounded_geometry,
            'properties': {'time': time_value}
        }
        combined_features.append(combined_feature)

# Create a new GeoDataFrame from the combined features
combined_gdf = gpd.GeoDataFrame.from_features(combined_features)
# print(combined_gdf)
combined_gdf.set_crs("EPSG:4326", inplace=True)
# Write the combined GeoDataFrame to a new GeoJSON file
combined_gdf.to_file(output_file_path, driver='GeoJSON')

print(f"Combined GeoJSON with intersections saved to {output_file_path}")


Combined GeoJSON with intersections saved to ../data/geojson/north_west/real/intersections/wc_202409166_1615.geojson
